In [1]:
import numpy as np
from MPS_code import *

### Make state vectors

In [131]:
num_of_qubits = 4
size_of_space = 2**num_of_qubits
state_vector_a = np.zeros(size_of_space, dtype=complex)
state_vector_b = np.zeros(size_of_space, dtype=complex)
state_vector_a[3] = 1
state_vector_b[4] = 1
state_vector_b[3] = 1
state_vector_b[-1] = 1
print('state a is ' + state_vectors_one_to_many(state_vector_a, as_str=True))
print('state b is ' + state_vectors_one_to_many(state_vector_b, as_str=True))

state a is 1.0+0.0j |0011>
state b is 1.0+0.0j |0011>+ 1.0+0.0j |0100>+ 1.0+0.0j |1111>


### Decompose into computational basis

In [132]:
state_vectors_one_to_many(state_vector_a)

array([[[ 1.+0.j,  0.+0.j],
        [ 1.+0.j,  0.+0.j],
        [ 0.+0.j,  1.+0.j],
        [ 0.+0.j,  1.+0.j]]])

### Create mps state from state vector or random and normalise

In [133]:
mps_a = create_specific_mps(state_vector_a)
a = normalise_mps(mps_a, direction='L')
mps_b = create_specific_mps(state_vector_b)
b = normalise_mps(mps_b, direction='L')
c = create_random_mps(num_of_qubits)

In [134]:
print('state a is ' + state_vectors_one_to_many(evaluate_mps(a), as_str=True))
print('state b is ' + state_vectors_one_to_many(evaluate_mps(b), as_str=True))
print('state c is ' + state_vectors_one_to_many(evaluate_mps(c), as_str=True))

state a is 1.0+0.0j |0011>
state b is 0.6+0.0j |0011>+ 0.6+0.0j |0100>+ 0.6+0.0j |1111>
state c is 0.0+0.3j |0000>+ 0.1+0.3j |0001>+ -0.0+0.2j |0010>+ 0.1+0.2j |0011>+ 0.0+0.3j |0100>+ 0.1+0.3j |0101>+ -0.0+0.2j |0110>+ 0.1+0.2j |0111>+ -0.0+0.3j |1000>+ 0.1+0.3j |1001>+ -0.0+0.2j |1010>+ 0.0+0.2j |1011>+ -0.0+0.3j |1100>+ 0.1+0.3j |1101>+ -0.0+0.2j |1110>+ 0.1+0.1j |1111>


### Find overlap and state amplitutes

In [135]:
find_overlap_of_mps(a, b)

(0.57735026918962573+0j)

In [136]:
evaluate_mps(b)

array([ 0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.57735027+0.j,
        0.57735027+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
        0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.57735027+0.j])

In [137]:
evaluate_mps(c, [0, 1, 1, 0])

(-0.013804910602883552+0.18707752422306842j)

### Compress mps

In [138]:
[i.shape for i in b]

[(2, 1, 2), (2, 2, 4), (2, 4, 2), (2, 2, 1)]

In [139]:
compressed_b = normalise_mps(b, direction='R', max_d=2)
print('state b is ' + state_vectors_one_to_many(evaluate_mps(compressed_b), as_str=True))
compressed_c = normalise_mps(c, direction='R', max_d=2)
print('state c is ' + state_vectors_one_to_many(evaluate_mps(compressed_c), as_str=True))

state b is -0.6+0.0j |0011>+ -0.6+0.0j |0100>+ -0.6+0.0j |1111>
state c is -0.0-0.3j |0000>+ -0.1-0.3j |0001>+ 0.0-0.2j |0010>+ -0.1-0.2j |0011>+ -0.0-0.3j |0100>+ -0.1-0.3j |0101>+ 0.0-0.2j |0110>+ -0.1-0.2j |0111>+ 0.0-0.3j |1000>+ -0.1-0.3j |1001>+ 0.0-0.2j |1010>+ -0.0-0.2j |1011>+ 0.0-0.3j |1100>+ -0.1-0.3j |1101>+ 0.0-0.2j |1110>+ -0.1-0.1j |1111>


In [140]:
[i.shape for i in compressed_b]

[(2, 1, 2), (2, 2, 2), (2, 2, 2), (2, 2, 1)]

In [141]:
abs(find_overlap_of_mps(c, compressed_c))

0.99999684324896432

In [142]:
find_overlap_of_mps(a, compressed_b)

(-0.57735026918962573+0j)

### Create Hamiltonian MPOs and find expectation values

In [143]:
mpo_mag = create_magnetisation_mpo(4, 0)

In [144]:
d = do_mpo_on_mps(mpo_mag, b)

In [145]:
print('state d is ' + state_vectors_one_to_many(evaluate_mps(d), as_str=True))

state d is 0.3+0.0j |0011>+ 0.3+0.0j |0100>+ -0.3+0.0j |1111>


In [146]:
abs(find_overlap_of_mps(b, d))

0.16666666666666671

In [149]:
mpo_heis = create_heisenberg_mpo(4, 2, 0.5, 3)

In [154]:
e = do_mpo_on_mps(mpo_heis, a)
print('state e is ' + state_vectors_one_to_many(evaluate_mps(e), as_str=True))

state e is 0.8+0.0j |0011>+ 1.0+0.0j |0101>


In [160]:
f = normalise_mps(e, direction='R')
print('state f is ' + state_vectors_one_to_many(evaluate_mps(f), as_str=True))

state f is 0.6+0.0j |0011>+ 0.8+0.0j |0101>


In [163]:
abs(find_overlap_of_mps(f, a))

0.59999999999999987

### Create Unitary MPOs and find expectation_values

In [166]:
mpo_heis_u = create_heisenberg_unitary_mpo(4, 2, 0, 0, 1)

In [167]:
e = do_mpo_on_mps(mpo_heis_u, a)

In [168]:
new_e = normalise_mps(e, direction='R')

In [23]:
print('state a is ' + state_vectors_one_to_many(evaluate_mps(a), as_str=True))

state a is 1.0+0.0j |011>


In [24]:
print('state new_e is ' + state_vectors_one_to_many(evaluate_mps(new_e), as_str=True))

state new_e is -0.8+0.0j |011>+ 0.6+0.0j |101>


In [25]:
abs(find_overlap_of_mps(new_e, new_e))

1.0000000000000004

In [26]:
np.sqrt(abs(find_overlap_of_mps(e, e)))

1.8027756377319946

In [27]:
abs(find_overlap_of_mps(e, e))

3.25

In [28]:
do_evolution(a, [mpo_heis_u], 5, max_d=8)

initial_state 1.0+0.0j |011>
0 state is -0.8+0.0j |011>+ 0.6+0.0j |101>
1 state is 0.7+0.0j |011>+ -0.7-0.0j |101>+ 0.2+0.0j |110>
2 state is 0.6+0.0j |011>+ -0.7-0.0j |101>+ 0.4+0.0j |110>
3 state is -0.6-0.0j |011>+ 0.7-0.0j |101>+ -0.4+0.0j |110>
4 state is -0.5-0.0j |011>+ 0.7-0.0j |101>+ -0.5+0.0j |110>
